In [1]:
import numpy as np, sympy as sp, scipy as scp
import scipy.integrate
from bokeh.plotting import figure, show, output_notebook

output_notebook()

Loading BokehJS ...

Найти численно: высота подъёма, время полёта (модель с сопротивлением).
Решить аналитически для вакуума.

* * *

Compute numerically the max elevation and the flight duration.
Solve analyticaclly for the model w/o resistance

In [2]:
def create_rhs(
                m = 0.009,    # kg
                g = 9.8,      # m/sec^2
                A = 1e-5,     # N*sec/m
                B = 1e-8,     # N*sec^3/m^3
            ):
    def rhs(zv, t):
        z, v = zv
        dzdt = v
        dvdt = -g - (A*v + B*v**3)/m # where the hell it comes from?
        return np.r_[dzdt, dvdt]
    return rhs

In [3]:
def solve(rhs,
          z0 = 0.0,    # m
          v0 = 500.0,  # m/sec
          t_max = 110  # sec
         ):
    T = np.linspace(0, stop=110, num=1000)
    ZV = scp.integrate.odeint(rhs, np.r_[z0, v0], T)
    actual_flight = ZV[:, 0] >= 0
    T = T[actual_flight]
    ZV = ZV[actual_flight, :]
    return T, ZV

In [4]:
def show_stats(title, T, ZV):
    print(title)
    print('max elevation: {max_el}\nflight time: {t}'.format(
        max_el=ZV[:, 0].max(),
        t=T.max()
    ))
    p = figure(plot_width=600, plot_height=400)
    p.line(T, ZV[:, 0], color='blue')
    p.line(T, ZV[:, 1], color='red')
    show(p)

In [5]:
rhs_drag = create_rhs()
rhs_no_drag = create_rhs(A=0, B=0)

In [6]:
T_drag, ZV_drag = solve(rhs_drag)
show_stats('model w resistance', T_drag, ZV_drag)

model w resistance
max elevation: 3466.325918020324
flight time: 52.632632632632635


In [7]:
T_no_drag, ZV_no_drag = solve(rhs_no_drag)
show_stats('model w/o resistance', T_no_drag, ZV_no_drag)

model w/o resistance
max elevation: 12755.094423753088
flight time: 101.96196196196196


In [8]:
p = figure()
p.line(T_drag, ZV_drag[:, 1], color='red')
p.line(T_no_drag, ZV_no_drag[:, 1], color='blue')
show(p)

# Errors

In [9]:
rhs, T, ZV = rhs_no_drag, T_no_drag, ZV_no_drag
expected_velo = [rhs(ZV[k, :], T[k])[0] for k, t in enumerate(T)]
expected_accel = [rhs(ZV[k, :], T[k])[1] for k, t in enumerate(T)]

In [10]:
np.abs(ZV[:, 1] - expected_velo).mean()

0.0

In [11]:
accel_devs = np.gradient(ZV[:, 1]) - expected_accel

In [12]:
np.abs(accel_devs).mean(), np.abs(accel_devs).max(),

(8.7209209209209195, 8.7209209209211629)

In [13]:
p = figure()
p.line(T, accel_devs)
show(p)

## Analytic solution for a simplified model